In [ ]:
sys.path.append('../')
import json

import requests
import pickle
import geopy.distance

import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt 
from mpl_toolkits.basemap import Basemap
import numpy as np 
# import swagger_client
import trassenfinder_route_request
from helpers.StationPhillip import StationPhillip
from helpers.BetriebsstellenBill import BetriebsstellenBill
from progress.bar import Bar
from time import sleep
import sqlalchemy
import sys

from config import db_database, db_password, db_server, db_username

import dask
import dask.dataframe as dd

In [ ]:
engine = sqlalchemy.create_engine('postgresql://'+ db_username +':' + db_password + '@' + db_server + '/' + db_database + '?sslmode=require') #

planned_path_df = pd.read_sql('SELECT pla_arr_path, pla_dep_path FROM rtd WHERE train_type <>\'Bus\'', con=engine)

In [ ]:
planned_path_df = pd.read_feather('../data_buffer/planned_path_df')

In [ ]:
planned_path_df.reset_index(drop=True).to_feather('../data_buffer/planned_path_df')

In [ ]:
planned_path_df = planned_path_df.sample(n=1000000).reset_index(drop=True) # 

In [ ]:
betriebstellen = BetriebsstellenBill()
stations = StationPhillip()
checked = pickle.load( open( "../checked_pickle", "rb" ) )
streckennetz = pickle.load( open( "../streckennetz3_pickle", "rb" ) )

In [ ]:
streckennetz = pickle.load( open( "../streckennetz3_pickle", "rb" ) )

In [ ]:
# These coordinates form the bounding box of Germany
left = 5.67
right = 15.64
bot = 47.06
top = 55.06
plt.figure(figsize=(180/2,100/2))
m = Basemap(llcrnrlon=left,llcrnrlat=bot,urcrnrlon=right,urcrnrlat=top,
             resolution='i', projection='tmerc', lat_0 = 51, lon_0 = 10)

m.drawcoastlines(linewidth=0.72, color='black')
m.drawcountries(zorder=0, color='black')
m.drawrivers(color='blue')

# xpt,ypt = m(stations.station_df['lon'].to_numpy(), stations.station_df['lat'].to_numpy())
# m.scatter(x=xpt, y=ypt, marker='.')

for (n1,n2) in streckennetz.edges(data=False):
    att_list = [att for att in streckennetz[n1][n2]]
    for att in att_list:
        if not att == 'distance':
            if att == 'length_km':
                streckennetz[n1][n2]['distance'] = streckennetz[n1][n2]['length_km']
            streckennetz[n1][n2].pop(att, None) 

positions = {}
betriebsstellen_without_location = []
for station in streckennetz.nodes():
    if not station:
        streckennetz.remove_node(station)
        # print("removed node")
    try:
        positions[station] = m(*stations.get_location(name=station))
    except KeyError:
        try:
            positions[station] = m(*stations.get_location(ds100=station))
        except KeyError:
            try:
                positions[station] = m(*betriebstellen.get_location(ds100=station))
            except betriebstellen.NoLocationError:
                betriebsstellen_without_location.append(station)
                # positions[station] = m(0.0,0.0)
            except KeyError:
                betriebsstellen_without_location.append(station)
                # positions[station] = m(0.0,0.0)
    # if not positions[station][0] or not positions[station][1]:
    #     del positions[station]

# for station in streckennetz.nodes():
#     positions[station] = m(0.0,0.0)
positions = nx.spring_layout(streckennetz, k=0.0001, pos=positions, fixed = positions.keys())
nx.draw_networkx(streckennetz, pos=positions) #nx.spring_layout(streckennetz))

plt.show()

In [ ]:
[(n1, n2) for (n1, n2) in streckennetz.edges()][0]

In [ ]:
streckennetz['Mönchengladbach Hbf']['Utrecht Centraal']

In [ ]:
g = nx.Graph()
g.add_edge('foo', 'bar')
g.add_edge(np.nan, np.nan)

In [ ]:
df = nx.to_pandas_edgelist(streckennetz)

In [ ]:
df = df.dropna(subset=['source', 'target']).reset_index(drop=True)
streckennetz = nx.from_pandas_edgelist(df)

In [ ]:
def get_paths_from_db():
    db_connect_string = 'postgresql://'+ db_username +':' + db_password + '@' + db_server + '/' + db_database + '?sslmode=require'
    planned_paths = dd.read_sql_table('rtd', db_connect_string, index_col='index', columns=['arr'], head_rows=10)
    return planned_paths.astype('Int64')

In [ ]:
meta_df = pd.DataFrame({'arr': pd.Series([], dtype='Int64')})

In [ ]:
df = get_paths_from_db()

In [ ]:
df = df.compute()

In [ ]:
df['arr']

In [ ]:
db_connect_string = 'postgresql://'+ db_username +':' + db_password + '@' + db_server + '/' + db_database + '?sslmode=require'
arr = pd.read_sql('SELECT arr FROM rtd', con=db_connect_string, coerce_float=False)

In [ ]:
arr.loc[:10, 'arr'] 

In [ ]:
arr